# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = "C:/Users/arc user/Desktop/python-api-challenge/WeatherPy/cities.csv"
weather_data = pd.read_csv(cities)
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,hermanus,-34.42,19.23,16.67,84,100,1.79,1602953193
1,1,deshnok,27.80,73.34,30.07,17,15,2.95,1602953193
2,2,tiksi,71.69,128.87,-6.67,98,100,4.34,1602953194
3,3,la presa,32.71,-117.00,24.44,52,5,1.76,1602953194
4,4,ballangen,68.34,16.83,3.33,69,100,7.20,1602953194
...,...,...,...,...,...,...,...,...,...
563,563,chubbuck,42.92,-112.47,14.00,44,20,10.30,1602953267
564,564,tucuma,-0.18,-50.05,32.98,48,89,4.41,1602953267
565,565,shakawe,-18.37,21.85,28.95,15,0,3.04,1602953267
566,566,nampula,-15.12,39.27,28.00,45,20,6.70,1602953082


Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
%matplotlib inline

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

#Store humidity
humidity = weather_data["Humidity"]



# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#A max temperature lower than 80 degrees but higher than 70 (converted to Celsius).
#Wind speed less than 10 mph.
#Zero cloudiness.
narrow_weather = weather_data.loc[(weather_data["Max Temp"]> 21.11)\
                 &(weather_data["Max Temp"]<26.66)&(weather_data["Wind Speed"]<10)&(weather_data["Cloudiness"]==0)].dropna()

                                                  

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrow_weather[["City", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]=""

In [8]:
hotel_df
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
hotel_df

,City,Lat,Lng,Hotel Name
17,san quintin,30.48,-115.95,Old Mill Hotel
25,jand,33.43,72.02,Mohalla Takia
96,richards bay,-28.78,32.04,BON Hotel Waterfront Richards Bay
235,gweru,-19.45,29.82,Suskwe Lodges
285,faanui,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
290,rosetta,31.40,30.42,Green Valley
302,hit,33.64,42.83,Aboody Alhachem
326,tura,25.52,90.22,Hotel RIKMAN Continental
330,gat,31.61,34.76,OlusHome
346,faya,18.39,42.45,مقبرة ال مجمل


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))